In [ ]:
import numpy as np
from numpy import genfromtxt
import glob
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import sys
pd.set_option('display.max_rows', 1000)
import random
from sklearn.utils import shuffle
from keras.callbacks import EarlyStopping
from scipy import stats
import math
from keras.models import Model

In [ ]:
def interpolate(df, index_list, col):
    count = len(index_list)
    
    before_index = index_list[0] - 1
    before_val = df.loc[before_index,col]
    
    after_index = index_list[len(index_list)-1] + 1
    after_val = df.loc[after_index,col]
    
    inc_amount = (after_val - before_val) / (count + 1)
    
    for i in range(count):
        update_index = index_list[i]
        update_val = df.loc[update_index-1,col] + inc_amount
        df.loc[update_index,col] = update_val
        
    return df

In [ ]:
data_folder = "data/Protocol/"
files_path = data_folder + "subject*.dat"

# ====================
#files_path = data_folder + "subject101.dat"
# ====================

df_main = pd.DataFrame(columns=['exp_id','label_id', \
                                'hand_acc0','hand_acc1','hand_acc2','hand_gyro0','hand_gyro1','hand_gyro2', \
                                'chest_acc0','chest_acc1','chest_acc2','chest_gyro0','chest_gyro1','chest_gyro2', \
                                'ankle_acc0','ankle_acc1','ankle_acc2','ankle_gyro0','ankle_gyro1','ankle_gyro2' \
                               ])

files = glob.glob(files_path)
for file in files:
    df_exp = pd.read_csv(file, header=None, delimiter=r"\s+")
    df_exp_filtered = df_exp.filter([1, 4, 5, 6, 10, 11, 12, 21, 22, 23, 27, 28, 29, 38, 39, 40, 44, 45, 46])
    df_exp_filtered.columns = ['label_id', \
                               'hand_acc0','hand_acc1','hand_acc2','hand_gyro0','hand_gyro1','hand_gyro2', \
                               'chest_acc0','chest_acc1','chest_acc2','chest_gyro0','chest_gyro1','chest_gyro2', \
                               'ankle_acc0','ankle_acc1','ankle_acc2','ankle_gyro0','ankle_gyro1','ankle_gyro2' \
                              ]
    
    exp_id = int(file[23])
    df_exp_filtered['exp_id'] = exp_id
    
    if exp_id==9:
        continue
    
    cols=['exp_id','label_id', \
          'hand_acc0','hand_acc1','hand_acc2','hand_gyro0','hand_gyro1','hand_gyro2', \
          'chest_acc0','chest_acc1','chest_acc2','chest_gyro0','chest_gyro1','chest_gyro2', \
          'ankle_acc0','ankle_acc1','ankle_acc2','ankle_gyro0','ankle_gyro1','ankle_gyro2' \
         ]
    df_exp_filtered = df_exp_filtered[cols]
    
    total_rows = len(df_exp_filtered)
    interpol_cols = ['hand_acc0','hand_acc1','hand_acc2','hand_gyro0','hand_gyro1','hand_gyro2', \
                     'chest_acc0','chest_acc1','chest_acc2','chest_gyro0','chest_gyro1','chest_gyro2', \
                     'ankle_acc0','ankle_acc1','ankle_acc2','ankle_gyro0','ankle_gyro1','ankle_gyro2' \
                    ]
    for col in interpol_cols:
        index = 0
        while index < total_rows:
            if math.isnan(df_exp_filtered.loc[index,col]):
                index_list = [index]
                nan_index = index + 1
                while (True):
                    if math.isnan(df_exp_filtered.loc[nan_index,col]):
                        index_list.append(nan_index)
                        nan_index += 1
                    else:
                        break
                df_exp_filtered = interpolate(df_exp_filtered, index_list, col)
            index += 1
    
    df_main = df_main.append(df_exp_filtered, ignore_index = True)

In [ ]:
# test interpolate for exp 1
# df_exp_filtered[19190:19199]

In [ ]:
window_size = 512
window_move_size = 100 #window_size // 2

In [ ]:
def trim(df_OVA):

    curr_index = 0
    act_start_index = 0
    act_end_index = 0

    df_OVA_trimmed = pd.DataFrame(columns=['exp_id','label_id', \
                                           'hand_acc0','hand_acc1','hand_acc2','hand_gyro0','hand_gyro1','hand_gyro2', \
                                           'chest_acc0','chest_acc1','chest_acc2','chest_gyro0','chest_gyro1','chest_gyro2', \
                                           'ankle_acc0','ankle_acc1','ankle_acc2','ankle_gyro0','ankle_gyro1','ankle_gyro2' \
                                          ])

    while (True):
        curr_act_id = df_OVA.loc[act_start_index, 'label_id']
    
        curr_index = act_start_index + 1
        while (True):
            act_id = df_OVA.loc[curr_index, 'label_id']
            if (act_id != curr_act_id):
                act_end_index = curr_index - 1
                break
            elif curr_index == (len(df_OVA) - 1):
                act_end_index = curr_index
                break
            else:
                curr_index += 1
            
        df_act_trimmed = df_OVA[act_start_index + 1000 : act_end_index - 1000]
    
        if len(df_act_trimmed) > 0:
            df_OVA_trimmed = df_OVA_trimmed.append(df_act_trimmed, ignore_index = True)
            #df_OVA_trimmed = df_OVA_trimmed.reset_index(drop=True)
    
        if curr_index >= (len(df_OVA) - 1):
            break
        else:
            act_start_index = act_end_index + 1
    
    return df_OVA_trimmed

In [ ]:
def segment(df_OVA):

    segmeneted_activities = [ 'N/A',[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[] ]

    for i in range(1 , 9):

        df_exp = df_OVA.loc[df_OVA['exp_id'] == i]
        df_exp = df_exp.reset_index(drop=True)

        index = 0
        continu = True
        while (continu):

            if index >= len(df_exp):
                break

            df_window = df_exp.loc[index:index+window_size-1]
            if len(df_window) < window_size:
                break

            window_label_ids = set(df_window['label_id'].values)
            if (len(window_label_ids) > 1) or \
               (len(window_label_ids)==1 and next(iter(window_label_ids))==0):
                index = index + window_move_size
                continue

            na_window = np.array((df_window.loc[:,'hand_acc0':'ankle_gyro2']))
            window_label_id = next(iter(window_label_ids))

            segmeneted_activities[window_label_id].append(na_window)
            index = index + window_move_size

    for i in range(1,25):
        segmeneted_activities[i] = np.array(segmeneted_activities[i])
        
    for i in range(1,25):
        print ("Act_id:" , i , " " , segmeneted_activities[i].shape)
        
    return segmeneted_activities

In [ ]:
def get_binary_X_and_Y_oversampled(act_id, segmeneted_activities):
    
    X , Y = get_binary_X_and_Y (act_id, segmeneted_activities)

    total_datapoints = X.shape[0]
    plus_datapoints = segmeneted_activities[act_id].shape[0]
    alpha = (total_datapoints - plus_datapoints) / plus_datapoints
    alpha = round(alpha, 1)
    fraction = int(str(alpha)[-1])
    alpha = int(alpha)
    plus_class_counter = -1
    
    New_X = []
    New_Y = []

    for i in range(total_datapoints):
        
        if (Y[i]==1):
            plus_class_counter += 1
            for j in range(alpha):
                New_X.append(X[i])
                New_Y.append(Y[i])

            if (plus_class_counter % 10) < fraction:
                New_X.append(X[i])
                New_Y.append(Y[i])
                
        else:
            New_X.append(X[i])
            New_Y.append(Y[i])

    New_X = np.array(New_X)
    New_Y = np.array(New_Y)
    
    return New_X , New_Y

In [ ]:
def get_binary_X_and_Y (act_id , segmeneted_activities):

    X = np.concatenate((segmeneted_activities[1], segmeneted_activities[2]))
    X = np.concatenate((           X            , segmeneted_activities[3]))
    X = np.concatenate((           X            , segmeneted_activities[17]))
    X = np.concatenate((           X            , segmeneted_activities[16]))
    X = np.concatenate((           X            , segmeneted_activities[12]))
    X = np.concatenate((           X            , segmeneted_activities[13]))
    X = np.concatenate((           X            , segmeneted_activities[4]))
    X = np.concatenate((           X            , segmeneted_activities[7]))
    X = np.concatenate((           X            , segmeneted_activities[6]))
    X = np.concatenate((           X            , segmeneted_activities[5]))
    X = np.concatenate((           X            , segmeneted_activities[24]))

    ys = ['N/A',None,None,None,None,None,None,None,None,None,None,None, \
          None,None,None,None,None,None,None,None,None,None,None,None,None]
    for i in range(1,25):
        size = segmeneted_activities[i].shape[0]
        if act_id == i:
            y = np.full (size , 1)
        else:
            y = np.full (size , 0)
        ys[i] = y

    Y = np.concatenate((ys[1], ys[2]))
    Y = np.concatenate((  Y  , ys[3]))
    Y = np.concatenate((  Y  , ys[17]))
    Y = np.concatenate((  Y  , ys[16]))
    Y = np.concatenate((  Y  , ys[12]))
    Y = np.concatenate((  Y  , ys[13]))
    Y = np.concatenate((  Y  , ys[4]))
    Y = np.concatenate((  Y  , ys[7]))
    Y = np.concatenate((  Y  , ys[6]))
    Y = np.concatenate((  Y  , ys[5]))
    Y = np.concatenate((  Y  , ys[24]))

    return (X , Y)

In [ ]:
def create_model (data_input_shape):

    LSTM_output_dim = 100
    
    model = Sequential()
    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM1', input_shape=data_input_shape))    
    #model.add(LSTM(LSTM_output_dim, return_sequences=True, name='LSTM2'))
    #model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM3'))
    
    model.add(LSTM(LSTM_output_dim, return_sequences=False, name='LSTM1', input_shape=data_input_shape))
    
    model.add(Dense(15, activation='sigmoid',name='Hidden_Layer'))
    
    #model.add(Dense(3, activation='softmax', name='Softmax_layer'))
    model.add(Dense(1, activation='sigmoid',name='Dense_out'))
    
    #model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
    model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

    return model

In [ ]:
def train_binary_classifier(act_id, segmeneted_activities):
    #(X , Y) = get_binary_X_and_Y (act_id , segmeneted_activities)
    (X , Y) = get_binary_X_and_Y_oversampled (act_id , segmeneted_activities)
    
    X , Y = shuffle(X , Y)

    X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25)
    X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.333)

    data_input_shape = X.shape[1:3]
    model = create_model(data_input_shape)
    #print (model.summary()  , "\n")

    patience = 5
    callbacks=[EarlyStopping(monitor='val_loss',patience=patience,verbose=1)]
    history = model.fit(X_train,y_train,
                        validation_data=[X_valid,y_valid],
                        epochs=20, 
                        batch_size=100,
                        verbose=2
                        ,callbacks=callbacks
                       )

    score = model.evaluate(X_test, y_test, verbose=0)
    print('\nTest loss:', score[0])
    print('Test accuracy:', score[1])
    print ()
    
    return model

In [ ]:
def train(df_OVA):
    df_OVA = trim(df_OVA)
    
    segmeneted_activities = segment(df_OVA)
    
    #training binary classifier for activity 1 (Lying)
    print ()
    print ("Training activity 1 ...")
    act_id = 1
    model_1 = train_binary_classifier(act_id, segmeneted_activities)
    #model_1.save('model512_1.h5')
    
    #training binary classifier for activity 2 (Sitting)
    print ("Training activity 2 ...")
    act_id = 2
    model_2 = train_binary_classifier(act_id, segmeneted_activities)
    #model_2.save('model512_2.h5')

    #training binary classifier for activity 3 (Standing)
    print ("Training activity 3 ...")
    act_id = 3
    model_3 = train_binary_classifier(act_id, segmeneted_activities)
    #model_3.save('model512_3.h5')
    
    #training binary classifier for activity 17 (Ironing)
    print ("Training activity 17 ...")
    act_id = 17
    model_17 = train_binary_classifier(act_id, segmeneted_activities)
    #model_17.save('model512_17.h5')
    
    #training binary classifier for activity 16 (Vacuum Cleaning)
    print ("Training activity 16 ...")
    act_id = 16
    model_16 = train_binary_classifier(act_id, segmeneted_activities)
    #model_16.save('model512_16.h5')
    
    #training binary classifier for activity 12 (Ascending Stairs)
    print ("Training activity 12 ...")
    act_id = 12
    model_12 = train_binary_classifier(act_id, segmeneted_activities)
    #model_12.save('model512_12.h5')
    
    #training binary classifier for activity 13 (Descending Stairs)
    print ("Training activity 13 ...")
    act_id = 13
    model_13 = train_binary_classifier(act_id, segmeneted_activities)
    #model_13.save('model512_13.h5')
    
    #training binary classifier for activity 4 (Walking)
    print ("Training activity 4 ...")
    act_id = 4
    model_4 = train_binary_classifier(act_id, segmeneted_activities)
    #model_4.save('model512_4.h5')
    
    #training binary classifier for activity 7 (Nordic Walking)
    print ("Training activity 7 ...")
    act_id = 7
    model_7 = train_binary_classifier(act_id, segmeneted_activities)
    #model_7.save('model512_7.h5')
    
    #training binary classifier for activity 6 (Cycling)
    print ("Training activity 6 ...")
    act_id = 6
    model_6 = train_binary_classifier(act_id, segmeneted_activities)
    #model_6.save('model512_6.h5')
    
    #training binary classifier for activity 5 (Running)
    print ("Training activity 5 ...")
    act_id = 5
    model_5 = train_binary_classifier(act_id, segmeneted_activities)
    #model_5.save('model512_5.h5')
    
    #training binary classifier for activity 24 (Rope Jumping)
    print ("Training activity 24 ...")
    act_id = 24
    model_24 = train_binary_classifier(act_id, segmeneted_activities)
    #model_24.save('model512_24.h5')
    
    models = [None,None,None,None,None,None,None,None,None,None,None,None, \
              None,None,None,None,None,None,None,None,None,None,None,None,None]
    models[1] = model_1
    models[2] = model_2
    models[3] = model_3
    models[17] = model_17
    models[16] = model_16
    models[12] = model_12
    models[13] = model_13
    models[4] = model_4
    models[7] = model_7
    models[6] = model_6
    models[5] = model_5
    models[24] = model_24

    return models

In [ ]:
# from keras.models import load_model
# model = load_model('model.h5')

In [ ]:
def predict(X_predict, models):

    X_predict = np.reshape(X_predict , (1,X_predict.shape[0],X_predict.shape[1]))

    scores = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

    for i in range(24):
        model = models[i+1]
        if (model is not None):
            pred_probs = model.predict_proba(X_predict)
            scores[i] = pred_probs[0][0]

    threshold = 0.5
    if max(scores) < threshold:
        label_id = 0
    else:
        label_id = scores.index(max(scores)) + 1
    
    return label_id, scores

In [ ]:
def test(df_continuous , models):

    # =================
    #df_continuous2 = df_continuous.loc[df_continuous['exp_id'] == 7]
    df_continuous2 = df_continuous
    # =================

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0
    while (continu):

        if index >= len(df_continuous2):
            break

        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break

        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]

        if window_label_id == 0:
            print ("Window Label: 0")
            index = index + window_move_size
            continue

        na_window = np.array((df_window.loc[:,'hand_acc0':'ankle_gyro2']))
        pred_label_id, _ = predict(na_window, models)

        print ("Window Label: " + str(window_label_id) + "  Predicted Label: " + str(pred_label_id))
        
        total_preds += 1

        if (pred_label_id == window_label_id):
            correct_preds += 1

        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

#### Cao et al Smoothing

In [ ]:
def get_u(df_OVA):

    run_lengths = []
    current_label = 0
    run_length = 1

    for i in range(1 , len(df_OVA)):
    
        label = df_OVA.loc[i , 'label_id']
    
        if current_label == label:
            run_length += 1
            continue
        else:
            if current_label != 0:
                run_lengths.append(run_length)
            run_length = 1
            current_label = label
            continue
        
    run_lengths = np.array(run_lengths)
    run_lengths_sorted = np.sort(run_lengths)
    p = 1. * np.arange(len(run_lengths)) / (len(run_lengths) - 1)

    #import matplotlib.pyplot as plt
    #plt.figure()
    #plt.plot(run_lengths_sorted, p)
    #plt.title('cumulative distribution function (CDF) of all run lengths')
    #plt.ylabel('p')
    #plt.xlabel('run length')
    #plt.show()

    thr = 0.03
    for i in range(p.shape[0]):
        if p[i] >= thr:
            break

    idx = i-1
    u = run_lengths_sorted[idx]
    u = round((u - window_size) / (2.0 * window_move_size))
    return u

In [ ]:
def test_with_cao_smoothing_3 (df_continuous , models):

    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0
    l = []
    w = []
    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            l.append(0)
            w.append(0)
        
        else:    
            na_window = np.array((df_window.loc[:,'hand_acc0':'ankle_gyro2']))
            pred_label_id, _ = predict(na_window, models)
            l.append(pred_label_id)
            w.append(window_label_id)
    
        i = len(l)
        if i == 2:
            if w[0] == 0:
                print ("Window Label: 0")
                index = index + window_move_size
                continue
            smoothed_pred_label_id = l[0]
            total_preds += 1
            if (smoothed_pred_label_id == w[0]):
                correct_preds += 1
        
        elif i > 2:
        
            if w[i-2]==0:
                print ("Window Label: 0")
                index = index + window_move_size
                continue
            
            elif w[i-1]==0:
                smoothed_pred_label_id = l[i-2]
            
            elif w[i-3]==0:
                smoothed_pred_label_id = l[i-2]
            
            else:
                pred_seg = [ l[i-3] , l[i-2] , l[i-1] ]
                mode = stats.mode(pred_seg)
                if mode[1][0] > 1:
                    smoothed_pred_label_id = mode[0][0]
                else:
                    smoothed_pred_label_id = l[i-2]
            
            print ("Window Label: " + str(w[i-2]) + "  Smoothed Predicted Label: " + str(smoothed_pred_label_id))
      
            total_preds += 1
        
            if (smoothed_pred_label_id == w[i-2]):        
                correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

In [ ]:
def test_with_cao_smoothing_5 (df_continuous , models):

    df_continuous2 = df_continuous
    
    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0
    l = []
    w = []
    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            l.append(0)
            w.append(0)
        
        else:    
            na_window = np.array((df_window.loc[:,'hand_acc0':'ankle_gyro2']))
            pred_label_id, _ = predict(na_window, models)
            l.append(pred_label_id)
            w.append(window_label_id)
    
        i = len(l)
        if i==3 or i==4:
            if w[i-3]==0:
                print ("Window Label: 0")
                index = index + window_move_size
                continue
            smoothed_pred_label_id = l[i-3]
            total_preds += 1
            if (smoothed_pred_label_id == w[i-3]):
                correct_preds += 1
        
        elif i >= 5:
        
            if w[i-3]==0:
                print ("Window Label: 0")
                index = index + window_move_size
                continue
            
            elif w[i-1]==0 or w[i-2]==0 or w[i-4]==0 or w[i-5]==0:
                smoothed_pred_label_id = l[i-3]
            
            else:
                pred_seg = [ l[i-5] , l[i-4] , l[i-3] , l[i-2] , l[i-1] ]
                mode = stats.mode(pred_seg)
                if mode[1][0] > 1:
                    smoothed_pred_label_id = mode[0][0]
                else:
                    smoothed_pred_label_id = l[i-3]
            
            print ("Window Label: " + str(w[i-3]) + "  Smoothed Predicted Label: " + str(smoothed_pred_label_id))
      
            total_preds += 1
        
            if (smoothed_pred_label_id == w[i-3]):        
                correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

#### Krishnan et al Smoothing

delta_t = 3

In [ ]:
def g(x):
    
    result = (1. / (math.sqrt(2*math.pi))) * np.exp(-0.5 * x * x)
    return result

In [ ]:
def get_window_feature_vector(X_predict , intermediate_layer_model):
    
    intermediate_output = intermediate_layer_model.predict(X_predict)
    intermediate_output = np.reshape(intermediate_output, (intermediate_output.shape[1]))
    
    return intermediate_output

In [ ]:
def euc_dist (f1 , f2):
    
    dist = np.linalg.norm(f2-f1)
    return dist

In [ ]:
def h(f1 , f2):

    alpha = 1
    result = np.exp(-alpha * euc_dist(f1 , f2))
    return result

In [ ]:
def get_predict_vector(na_window, models, intermediate_layer_models):
    
    X_predict = na_window
    X_predict = np.reshape(X_predict , (1,X_predict.shape[0],X_predict.shape[1]))

    scores = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    f = [None, None, None, None, None, None, None, None, None, None, None, None, \
         None, None, None, None, None, None, None, None, None, None, None, None]
    
    for i in range(24):
        model = models[i+1]
        if (model is not None):
            pred_probs = model.predict_proba(X_predict)
            scores[i] = pred_probs[0][0]

            f[i] = get_window_feature_vector(X_predict , intermediate_layer_models[i+1])
    
    return scores , f

In [ ]:
def test_with_Krishnan_smoothing_delta3(df_continuous , models):

    intermediate_layer_model_1 = Model(inputs=models[1].input, outputs=models[1].get_layer("LSTM1").output)
    intermediate_layer_model_2 = Model(inputs=models[2].input, outputs=models[2].get_layer("LSTM1").output)
    intermediate_layer_model_3 = Model(inputs=models[3].input, outputs=models[3].get_layer("LSTM1").output)
    intermediate_layer_model_17 = Model(inputs=models[17].input, outputs=models[17].get_layer("LSTM1").output)
    intermediate_layer_model_16 = Model(inputs=models[16].input, outputs=models[16].get_layer("LSTM1").output)
    intermediate_layer_model_12 = Model(inputs=models[12].input, outputs=models[12].get_layer("LSTM1").output)
    intermediate_layer_model_13 = Model(inputs=models[13].input, outputs=models[13].get_layer("LSTM1").output)
    intermediate_layer_model_4 = Model(inputs=models[4].input, outputs=models[4].get_layer("LSTM1").output)
    intermediate_layer_model_7 = Model(inputs=models[7].input, outputs=models[7].get_layer("LSTM1").output)
    intermediate_layer_model_6 = Model(inputs=models[6].input, outputs=models[6].get_layer("LSTM1").output)
    intermediate_layer_model_5 = Model(inputs=models[5].input, outputs=models[5].get_layer("LSTM1").output)
    intermediate_layer_model_24 = Model(inputs=models[24].input, outputs=models[24].get_layer("LSTM1").output)

    intermediate_layer_models = [None,None,None,None,None,None,None,None,None,None,None,None, \
                                 None,None,None,None,None,None,None,None,None,None,None,None,None]
    intermediate_layer_models[1] = intermediate_layer_model_1
    intermediate_layer_models[2] = intermediate_layer_model_2
    intermediate_layer_models[3] = intermediate_layer_model_3
    intermediate_layer_models[17] = intermediate_layer_model_17
    intermediate_layer_models[16] = intermediate_layer_model_16
    intermediate_layer_models[12] = intermediate_layer_model_12
    intermediate_layer_models[13] = intermediate_layer_model_13
    intermediate_layer_models[4] = intermediate_layer_model_4
    intermediate_layer_models[7] = intermediate_layer_model_7
    intermediate_layer_models[6] = intermediate_layer_model_6
    intermediate_layer_models[5] = intermediate_layer_model_5
    intermediate_layer_models[24] = intermediate_layer_model_24
    
    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0

    num_of_classes = 24
    P = [[] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , \
         [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , []]
    fs = [[] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , \
         [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , []]
    delta_t = 3

    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            P = [[] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , \
                 [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , []]
            fs = [[] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , \
                  [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , []]
            print ("Window Label: 0")
            index = index + window_move_size
            continue
        
        na_window = np.array((df_window.loc[:,'hand_acc0':'ankle_gyro2']))
        p , f = get_predict_vector(na_window , models, intermediate_layer_models)
    
        t = len(P[0])
    
        if t < (delta_t):
            for i in range(num_of_classes):
                P[i].append(p[i])
                fs[i].append(f[i])
            
        else:
            for i in range(num_of_classes):
                
                if (i+1) in [1,2,3,17,16,12,13,4,7,6,5,24]:
            
                    fs[i].append(f[i])
            
                    s1 = 0
                    for j in range(1 , delta_t+1):
                        s1 += g(j) * h(fs[i][t-j] , fs[i][t]) * P[i][t-j]
            
                    s2 = 0
                    for j in range(1 , delta_t+1):
                        s2 += g(j) * h(fs[i][t-j] , fs[i][t])
            
                    P[i].append( (p[i] + s1) / (s2 + 1) )
                
                else:
                    P[i].append(0)
    
    
        current_P = []
        for i in range(num_of_classes):
            current_P.append ( P[i][t] )
    
        threshold = 0.5
        if max(current_P) < threshold:
            pred_label_id = 0
        else:
            pred_label_id = current_P.index(max(current_P)) + 1
    
        print ("Window Label: " + str(window_label_id) + "  Smoothed Predicted Label: " + str(pred_label_id))
    
        total_preds += 1
        
        if (pred_label_id == window_label_id):
            correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

##### Simple Voting Scheme

N = 3

In [ ]:
def test_with_simple_voting_smoothing(df_continuous , models):

    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0

    N = 3
    l = []

    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            l = []
            print ("Window Label: 0")
            index = index + window_move_size
            continue
        
        na_window = np.array((df_window.loc[:,'hand_acc0':'ankle_gyro2']))
        pred_label_id, _ = predict(na_window, models)
        l.append(pred_label_id)

        t = len(l)
    
        if t < N:
            smoothed_pred_label_id = pred_label_id
            
        else:
            voting_list = []
            for i in range(N):
                voting_list.append( l[t-i-1] )
        
            mode = stats.mode(voting_list)
        
            if mode[1][0] > 1:
                smoothed_pred_label_id = mode[0][0]
            else:
                smoothed_pred_label_id = pred_label_id

    
        print ("Window Label: " + str(window_label_id) + "  Smoothed Predicted Label: " + str(smoothed_pred_label_id))
    
        total_preds += 1
        
        if (smoothed_pred_label_id == window_label_id):        
            correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

##### Exponentially Weighted Voting

alpha = 0.05, 0.25

In [ ]:
def test_with_EWV_smoothing (df_continuous , models, alpha):

    df_continuous2 = df_continuous

    index = 0
    continu = True
    correct_preds = 0
    total_preds = 0

    num_of_classes = 25
    w = [ [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , \
          [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] ]

    while (continu):
        
        if index >= len(df_continuous2):
            break
        
        df_window = df_continuous2.loc[index:index+window_size-1]
        if len(df_window) < window_size:
            break
        
        mode_win_labels = stats.mode(df_window['label_id'].values)
        window_label_id = mode_win_labels[0][0]
    
        if window_label_id == 0:
            w = [ [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , \
                  [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] , [] ]
            print ("Window Label: 0")
            index = index + window_move_size
            continue
        
        na_window = np.array((df_window.loc[:,'hand_acc0':'ankle_gyro2']))
        pred_label_id, _ = predict(na_window, models)

        t = len(w[0])
    
        if t == 0:
            smoothed_pred_label_id = pred_label_id
            for i in range(num_of_classes):
                if i==pred_label_id:
                    w[i].append(alpha)
                else:
                    w[i].append(0)
            
        else:
            current_ws = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            for i in range(num_of_classes):
            
                if i == pred_label_id:
                    c = 1
                else:
                    c = 0
                weight = w[i][t-1] + alpha * ( c - w[i][t-1])
                w[i].append(weight)
                current_ws[i] = weight
        
            smoothed_pred_label_id = current_ws.index(max(current_ws))

    
        print ("Window Label: " + str(window_label_id) + "  Smoothed Predicted Label: " + str(smoothed_pred_label_id))
    
        total_preds += 1
        
        if (smoothed_pred_label_id == window_label_id):        
            correct_preds += 1
        
        index = index + window_move_size
            
    accu = correct_preds/total_preds
    print ("\nContinuous Accuracy: " + str(accu))
    return accu

In [ ]:
folds_accu = [0,0,0,0,0,0,0,0]
folds_cao_3_accu = [0,0,0,0,0,0,0,0]
folds_cao_5_accu = [0,0,0,0,0,0,0,0]
folds_krishnan_accu = [0,0,0,0,0,0,0,0]
folds_simple_voting_accu = [0,0,0,0,0,0,0,0]
folds_EWV_alpha_5_accu = [0,0,0,0,0,0,0,0]
folds_EWV_alpha_25_accu = [0,0,0,0,0,0,0,0]
#folds_EWV_alpha_95_accu = [0,0,0,0,0,0,0,0]

for i in range(1,9):
    split_exp_id = i
    df_continuous = df_main.loc[df_main['exp_id'] == split_exp_id]
    df_continuous = df_continuous.reset_index(drop=True)

    df_OVA = df_main.loc[df_main['exp_id'] != split_exp_id]
    df_OVA = df_OVA.reset_index(drop=True)

    print ("fold" , i , "Training:")
    print ("len of df_main:" , str(len(df_main)))
    print ("len of df_continuous:" , str(len(df_continuous)))
    print ("len of df_OVA:" , str(len(df_OVA)))
    print ()
    
    models = train(df_OVA)
    
    print ("\nfold" , i , "Testing:")
    accu = test(df_continuous, models)
    folds_accu[i-1] = accu
    
    print ("\nCao et al Smoothing (length=3):")
    cao_3_accu = test_with_cao_smoothing_3 (df_continuous , models)
    folds_cao_3_accu[i-1] = cao_3_accu
    
    print ("\nCao et al Smoothing (length=5):")
    cao_5_accu = test_with_cao_smoothing_5 (df_continuous , models)
    folds_cao_5_accu[i-1] = cao_5_accu
    
    print ("\nKrishnan et al Smoothing (delta_t=3):")
    krishnan_accu = test_with_Krishnan_smoothing_delta3(df_continuous , models)
    folds_krishnan_accu[i-1] = krishnan_accu
    
    print ("\nSimple Voting Scheme Smoothing:")
    simple_voting_accu = test_with_simple_voting_smoothing(df_continuous , models)
    folds_simple_voting_accu[i-1] = simple_voting_accu
    
    print ("\nExponentially Weighted Voting (EWV) Smoothing (alpha=0.05):")
    EWV_alpha_5_accu = test_with_EWV_smoothing (df_continuous , models, 0.05)
    folds_EWV_alpha_5_accu[i-1] = EWV_alpha_5_accu

    print ("\nExponentially Weighted Voting (EWV) Smoothing (alpha=0.25):")
    EWV_alpha_25_accu = test_with_EWV_smoothing (df_continuous , models, 0.25)
    folds_EWV_alpha_25_accu[i-1] = EWV_alpha_25_accu

    #print ("\nExponentially Weighted Voting (EWV) Smoothing (alpha=0.95):")
    #EWV_alpha_95_accu = test_with_EWV_smoothing (df_continuous , models, 0.95)
    #folds_EWV_alpha_95_accu[i-1] = EWV_alpha_95_accu
    
    print()

In [ ]:
def get_avg_folds (lst):
    sum = 0
    for i in range(len(lst)):
        sum += lst[i]
    
    avg = sum / len(lst)
    return avg

In [ ]:
avg_accu = get_avg_folds(folds_accu)
print ("\nAverage Accuracy for all folds:" , avg_accu)

avg_cao_3_accu = get_avg_folds(folds_cao_3_accu)
print ("\nAverage Accuracy with Cao Smoothing (length=3) for all folds:" , avg_cao_3_accu)

avg_cao_5_accu = get_avg_folds(folds_cao_5_accu)
print ("\nAverage Accuracy with Cao Smoothing (length=5) for all folds:" , avg_cao_5_accu)

avg_krishnan_accu = get_avg_folds(folds_krishnan_accu)
print ("\nAverage Accuracy with Krishnan Smoothing (delta_t=3) for all folds:" , avg_krishnan_accu)

avg_simple_voting_accu = get_avg_folds(folds_simple_voting_accu)
print ("\nAverage Accuracy with Simple Voting Scheme Smoothing for all folds:" , avg_simple_voting_accu)

avg_EWV_alpha_5_accu = get_avg_folds(folds_EWV_alpha_5_accu)
print ("\nAverage Accuracy with EWV Smoothing (alpha=0.05) for all folds:" , avg_EWV_alpha_5_accu)

avg_EWV_alpha_25_accu = get_avg_folds(folds_EWV_alpha_25_accu)
print ("\nAverage Accuracy with EWV Smoothing (alpha=0.25) for all folds:" , avg_EWV_alpha_25_accu)